In [0]:
#import necessary library 
import os
from pathlib import Path
import torch
from torch.utils.data import TensorDataset ,DataLoader
from torch import nn,optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from torchvision import datasets,transforms





In [85]:
# Download and load the training data


trainset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(),download=True)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=150, shuffle=True)
testset=datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())
test_loader = torch.utils.data.DataLoader(testset, batch_size=150, shuffle=False)
print(f"len of trainSet {len(trainset)} , len of testSet {len(testset)}")

len of trainSet 60000 , len of testSet 10000


In [0]:
#Build a Network
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 95)
        self.fc3 = nn.Linear(95, 64)
       
        self.fc4 = nn.Linear(64, 10)

        # Dropout module with 0.2 drop probability
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)

        # Now with dropout
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        

        # output so no dropout here
        x = F.log_softmax(self.fc4(x), dim=1)

        return x
        
model=Network()

optimizer=optim.SGD(model.parameters(),lr=0.01,weight_decay=1e-6,momentum=0.9,nesterov=True)
criterion=nn.NLLLoss()

In [97]:
#Train the Data

epochs=10
train_losses,test_losses=[],[]
for e in range(epochs):
    running_loss=0
    for images,labels in train_loader:
        optimizer.zero_grad()
        log_ps=model(images)
        loss=criterion(log_ps,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
        
    else:
        test_loss=0
        accuracy=0
        
        with torch.no_grad():
            model.eval()
            for images,labels in test_loader:
                log_ps=model(images)
                test_loss+=criterion(log_ps,labels)
                ps=torch.exp(log_ps)
                top_p,top_class=ps.topk(1,dim=1)
                equals=top_class==labels.view(*top_class.shape)
                accuracy+=torch.mean(equals.type(torch.FloatTensor))
        model.train()
        train_losses.append(running_loss/len(train_loader))
        test_losses.append(test_loss/len(test_loader))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(train_loader)),
              "Test Loss: {:.3f}.. ".format(test_loss/len(test_loader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(test_loader))) 

Epoch: 1/10..  Training Loss: 1.364..  Test Loss: 0.396..  Test Accuracy: 0.885
Epoch: 2/10..  Training Loss: 0.417..  Test Loss: 0.246..  Test Accuracy: 0.926
Epoch: 3/10..  Training Loss: 0.287..  Test Loss: 0.177..  Test Accuracy: 0.947
Epoch: 4/10..  Training Loss: 0.229..  Test Loss: 0.148..  Test Accuracy: 0.953
Epoch: 5/10..  Training Loss: 0.193..  Test Loss: 0.128..  Test Accuracy: 0.961
Epoch: 6/10..  Training Loss: 0.170..  Test Loss: 0.112..  Test Accuracy: 0.965
Epoch: 7/10..  Training Loss: 0.150..  Test Loss: 0.105..  Test Accuracy: 0.968
Epoch: 8/10..  Training Loss: 0.139..  Test Loss: 0.097..  Test Accuracy: 0.971
Epoch: 9/10..  Training Loss: 0.127..  Test Loss: 0.092..  Test Accuracy: 0.973
Epoch: 10/10..  Training Loss: 0.117..  Test Loss: 0.086..  Test Accuracy: 0.974


In [98]:
#save our model
print("Our model: \n\n", model, '\n')
print("The state dict keys: \n\n", model.state_dict().keys())

Our model: 

 Network(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=95, bias=True)
  (fc3): Linear(in_features=95, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
) 

The state dict keys: 

 odict_keys(['fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias', 'fc4.weight', 'fc4.bias'])


In [99]:
#check number of parameters
pytorch_total_params = sum(p.numel() for p in model.parameters())
pytorch_total_params

119529